# Summer air conditioning: cooling & duhumidification
# Graphical psychrometric analysis

## 1. Problem

An air-conditioning system, composed of a cooling coil an recirculated air,
controls the indoor temperature at 26.0 °C and relative humidity at
50 %.

The outdoor conditions are: temperature 32.0 °C and relative humidity
50 %.

The mass flow rate of outdoor air is 1.00 kg/s.

The overall heat transmitance (U-value) of the building is
675 W/K and the mass flow rate of infiltration air is
1.35 kg/s.

The auxiliary sensible and latent loads of the thermal zone are
34.0 kW and 4.0 kW respectively.

<img src="Figures/Quiz.png" alt="quiz.png" style="width: 600px;"/>

> Fig. 1. Screenshot of the Moodle quiz with filled in answers.

In [20]:
%reset -f

import numpy as np
import psychro as psy

# constants
c = 1e3         # J/kg K, air specific heat
l = 2496e3      # J/kg, latent heat

# Given data
θI, φI = 26, 0.5            # °C, %, indoor air
θo, φo = 32, 0.5            # °C, %, outdoor air
mo = 1.00                   # kg/s, outdoor (fresh) air
US, mi = 675, 1.35          # W/K, kg/s, overall conductiviti, infiltration
Qsa, Qla = 34_000, 4_000    # W, W, sensible, latent auxiliary heat

## 2. By-pass factor of the cooling coil is zero

**Find:**

- supply air mass flow rate
{1:NUMERICAL:=3.00:0.2} ±0.2 kg/s ;

- total load of the cooling coil
{1:NUMERICAL:=-82.30:5} ±5 kW ;

- apparatus dew point (ADP), i.e. the equivalent surface temperature of the
cooling coil {1:NUMERICAL:=10.6:1.5} ±1.5 °C.


---------------------------

<img src="Figures/Diag_beta_0.pdf" alt="Diag_beta_0.pdf" style="width: 600px;"/>

> Fig. 2.a Bloc diagram of the model of the HVAC system: no by-pass, $\beta = 0$.

**Solution**

Find the sensible and latent loads of the thermal zone, $\dot{Q}_{sTZ}$ and $\dot{Q}_{lTZ}$: 

In [2]:
wo = psy.w(θo, φo)
wI = psy.w(θI, φI)

QsTZ = (US + mi * c) * (θo - θI) + Qsa  # W, sensible load thermal zone
QlTZ = mi * l * (wo - wI) + Qla         # W, latent load thermal zone

Find the intersection of supply air line (TZ-line) with the saturation curve:

$$\dot{m} c\theta_s = \dot{m} c \theta_I + Q_{sTZ}$$
$$\dot{m} l w_s = \dot{m} l w_I + Q_{lTZ}$$
$$\dot{m} c\theta_s + Q_{sCC}= \dot{m} c \theta_I$$
$$\dot{m} l w_s + Q_{lCC}= \dot{m} l w_I$$
$$w' \theta_s - w_s(\theta_s) = w' \theta_0 - w_s(\theta_0)$$
$$Q_{sCC} + Q_{lCC} - Q_{tCC} = 0$$

6 equations, 6 unknowns: $\dot{m}, \theta_s, w_s, Q_{sCC}, Q_{lCC}, Q_{tCC}$

Graphical solution:

Search point *s* on the pshychrometric chart.
Draw the supply air line (for the thermal zone) by finding a point (R) on the psychromeric chart:

In [3]:
θR = 40
wR = wI + QlTZ / QsTZ * c / l * (θR - θI)
print(f'(I): {θI:.1f} °C, {1000 * wI:.1f} g/kg')
print(f'(R): {θR:.1f} °C, {1000 * wR:.1f} g/kg')

(I): 26.0 °C, 10.5 g/kg
(R): 40.0 °C, 12.8 g/kg


<img src="Figures/PsyC_supply_line.pdf" alt="PsyC_supply_line.pdf" style="width: 600px;"/>

> Fig. 2a. The intersection between the line of the supply line (thermal zone) and the saturation curve gives the Apparatus Dew Point (ADP) : 10.6 °C, 7.9 g/kg

In [4]:
θs = 10.6   # °C, from psychrometric chart
print(f'Apparatus Dew Point (ADP) {θs:.1f} °C, {1000 * psy.w(θs, 1):.2f} g/kg')

Apparatus Dew Point (ADP) 10.6 °C, 7.95 g/kg


In [5]:
m = QsTZ / (c * (θI - θs))
print(f'Mass flow rate of supply air: {m:.2f} kg/s')

Mass flow rate of supply air: 3.00 kg/s


In [6]:
mr = m - mo     # recirculated air
θM = (mo * θo + mr * θI) / m
wM = (mo * wo + mr * wI) / m
print(f'Mixing point: {θM:.1f} °C, {1000 * wM:.2f} g/kg')

Mixing point: 28.0 °C, 11.98 g/kg


In [7]:
QtCC = m * (c * (θs - θM) + l * (psy.w(θs, 1) - wM))
print(f'Total load of the cooling coil: {QtCC / 1000:.2f} kW')

Total load of the cooling coil: -82.35 kW


<img src="Figures/PsyC_beta_0.pdf" alt="PsyC_beta0.pdf" style="width: 600px;"/>

> Fig. 2b. Representation of processes on the psychrmetric chart

## 3. By-pass factor of the cooling coil is 0.16

Find:

- supply air mass flow rate
{1:NUMERICAL:=3.53:0.2} ±0.2 kg/s ;

- total load of the cooling coil
{1:NUMERICAL:=-82.31:5} ±5 kW ;

- apparatus dew point (ADP), i.e. the equivalent surface temperature of the
cooling coil {1:NUMERICAL:=10.1:1.5} ±1.5 °C.

-----------------

<img src="Figures/Diag_beta_016.pdf" alt="Diag_beta_016.pdf" style="width: 600px;"/>

> Fig. 3.a Bloc diagram of the model of the HVAC system: fixed by-pass factor $\beta \neq 0$. 

**Solution**

Since there is a by-pass, we expect the mass flow rate through the AHU, $\dot{m}$, to be larger. Since the mass flow rate of fresh air, $\dot{m}_o$, does not change, the mixing point, *M*, will be nearer the indoor point, *I*, than in the case of $\beta = 0$. The by-pass air flow will be recirculated without changing the indoor condition. Let's make a guess:

In [8]:
β = 0.16

m *= 1.16       # guess: mass flow rate increses by 16%
mr = m - mo     # recirculated air
θM = (mo * θo + mr * θI) / m
wM = (mo * wo + mr * wI) / m
print(f'Mixing point: {θM:.1f} °C, {1000 * wM:.2f} g/kg')

Mixing point: 27.7 °C, 11.78 g/kg


Procedure:

<img src="Figures/PsyC_beta_016a.pdf" alt="PsyC_beta_016a.pdf" style="width: 600px;"/>

> Fig. 3.b Draw a paralel to TZ-line that goes through the mixing point, *M*. Measure $\Delta w$ betweent *M* and the TZ-line (on red). 

In [9]:
w1 = 13.75e-3
Δw = w1 - wR

The CC-line (E - s) needs to be cut by the parallels in teh ratio β:
$$\beta = \frac{w_R - w_2}{w_1 - w_2}$$

So,
$$w_2 = \frac{w_R - \beta w_1}{1 - \beta}$$

In [10]:
w2 = (wR - β * w1) / (1 - β)
print(f'w2 = {1000 * w2:.2f} g/kg')

w2 = 12.63 g/kg


<img src="Figures/PsyC_beta_016b.pdf" alt="PsyC_beta_016b.pdf" style="width: 600px;"/>

> Fig. 3.c Draw a parallel to TZ-line that passes through point *2*. The ADP is at the intersection of this line with the saturation curve.

<img src="Figures/PsyC_beta_016c.pdf" alt="PsyC_beta016c.pdf" style="width: 600px;"/>

> Fig. 3.d The supply air, *S*, is at the crosing of TZ-line (black) and CC-line (blue).

In [11]:
#  From psychrometric chart
θS = 12.8     # °C
wS = 8.3e-3   # kg/kg

In [12]:
m = QsTZ / (c * (θI - θS))
print(f'Supply air mass flow rate: {m:.2f} kg/s')

Supply air mass flow rate: 3.50 kg/s


The initial guess was $\dot{m} = 1.16$ kg/s. Check if the initial guess of the supply air mass flow rate was good:

In [13]:
print(f'The ratio betwen the mass flow rate with by-pass and without by-pass is {m / 3.00:.2f}')

The ratio betwen the mass flow rate with by-pass and without by-pass is 1.17


In [14]:
QtCC = m * (c * (θS - θM) + l * (wS - wM))
print(f'Total load of the cooling coil: {QtCC / 1000:.2f} kW')

Total load of the cooling coil: -82.55 kW


## 4. The supply air mass flow rate is 3.50 kg/s

Find:

- the by-pass factor
{1:NUMERICAL:=0.15:0.05} ±0.05;

- the total load of the cooling coil
{1:NUMERICAL:=-82.31:5} ±5 kW;

- apparatus dew point (ADP), i.e. the equivalent surface temperature of the
cooling coil {1:NUMERICAL:=10.1:1.5} ±1.5 °C.

-----------------

<img src="Figures/Diag_m_35.pdf" alt="Diag_m_35.pdf" style="width: 600px;"/>

> Fig. 4.a Bloc diagram of the model of the HVAC system: fixed supply air mass flow rate $\dot{m}$. 

**Solution**

In [15]:
m = 3.5
θS = θI - QsTZ / (m * c)
wS = wI - QlTZ / (m * l)
print(f'Supply air temperature {θS:.1f} °C')

Supply air temperature 12.8 °C


In [16]:
mr = m - mo     # recirculated air
θM = (mo * θo + mr * θI) / m
wM = (mo * wo + mr * wI) / m
print(f'Mixing point: {θM:.1f} °C, {1000 * wM:.2f} g/kg')

Mixing point: 27.7 °C, 11.77 g/kg


Apparatus Dew Point (ADP) is on the saturation curve at the intersectio with the line of the Coling Coil which passses through mixing (M) and supply (S) points. 

<img src="Figures/PsyC_m_35a.pdf" alt="PsyC_m_35a.pdf" style="width: 600px;"/>

> Fig. 4.b Representation on the psychrometric chart

In [17]:
θs = 10.1   # °C, from psychrometric chart
print(f'Apparatus Dew Point (ADP): {θs:.1f} °C, {1000 * psy.w(θs, 1):.2f} g/kg')

Apparatus Dew Point (ADP): 10.1 °C, 7.68 g/kg


In [18]:
β = (θS - θs) / (θM - θs)
print(f'By-pass factor {β:.2f}')

By-pass factor 0.15


In [19]:
QtCC = m * (c * (θS - θM) + l * (wS - wM))
print(f'Total load of the cooling coil: {QtCC / 1000:.2f} kW')

Total load of the cooling coil: -82.31 kW


# Annex

<img src="Figures/Elem_process.pdf" alt="Elem_process.pdf" style="width: 600px;"/>

> Fig. A. Elementary psychrometric processes.